In [ ]:
import os, sys
import numpy as np
import pandas as pd
import astropy.io.fits as pyfits

# Plotting
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from astropy.visualization import MinMaxInterval, AsinhStretch, ImageNormalize
import matplotlib.image as mpimg
from mpl_toolkits.axes_grid1 import make_axes_locatable
from tqdm import tqdm_notebook
import lenstronomy.Util.param_util as param_util

# H0rton
from scipy.stats import uniform, norm
import h0rton.tdlmc_data
import h0rton.tdlmc_utils
import h0rton.h0_inference

%matplotlib inline
%load_ext autoreload
%autoreload 2

## Verifying the inference method on Rung 1 data

__Author:__ Ji Won Park (@jiwoncpark) and Joshua Lin (@joshualin24)

__Created:__ 9/10/2019

__Last run:__ 10/17/2019

__Goals:__
We verify the inference method \using the Rung 1 data. By inputting true mass model parameters, we should be able to recover the true H0 when we sample from the H0 posterior.

__Before_running:__
1. Download and extract the TDLMC data by running `download_tdlmc_data.sh`, e.g. at the root of the repo,
```bash
source h0rton/tdlmc_data/download_tdlmc_data.sh
```
2. Combine the Rung 1 data into a csv file by running, in a Python environment,
```python
import h0rton.tdlmc_utils
h0rton.tdlmc_utils.convert_to_dataframe(rung=1, save_csv_path=None)
```

In [ ]:
tdlmc_data_path = os.path.abspath(h0rton.tdlmc_data.__path__[0])
rung1_combined_path = os.path.join(tdlmc_data_path, 'rung1_combined.csv')
df = h0rton.tdlmc_utils.read_from_csv(rung1_combined_path)

Let us sample from the H0 posterior of one lens as an example.

In [ ]:
example_lens = df.iloc[0]
example_lens

Here is what a drizzled image of this lens looks like. Pay attention to the ordering of the AGN images.

In [ ]:
seed_path = example_lens['seed_path']
drizzled_img_path = os.path.join(seed_path, 'drizzled_image', 'lens-image.fits')
abcd_path = os.path.join(seed_path, 'ABCD.png')

hdul = pyfits.open(drizzled_img_path)
print(hdul.info())
img = hdul['PRIMARY'].data
abcd = plt.imread(abcd_path)

plt.subplots(3, figsize=(20, 60))

plt.subplot(1, 3, 1)
asinh_norm = ImageNormalize(img, interval=MinMaxInterval(), stretch=AsinhStretch())
plt.imshow(img, origin='lower left', norm=asinh_norm)
plt.title("log scale")

plt.subplot(1, 3, 2)
plt.imshow(img,  origin='lower left')
plt.title("regular scale")

plt.subplot(1, 3, 3)
plt.imshow(np.flipud(abcd),  origin='lower left')
plt.title("ABCD")
plt.axis('off')

plt.tight_layout()
plt.show()


The `h0rton.h0_inference.H0Posterior` class can be instantiated as follows. Note the minor but important differences in parameter definitions between `lenstronomy` and TDLMC. The `abcd_ordering` (the ordering of A, B, C, D images in increasing dec) is a keyword argument that must be entered manually for each lens.

The cell below defines the point source in terms of the source positions. As this quantity is the quantity that's sampled by the TDLMC evil team, it's more precise than using the published image positions (subject to rounding errors).

In [ ]:
e1_lens, e2_lens = param_util.phi_q2_ellipticity(q=example_lens['lens_mass_q'], 
                                                 phi=example_lens['lens_mass_phi_G'])
kwargs_spemd = {'theta_E': example_lens['lens_mass_theta_E'],
                'center_x': 0, 
                'center_y': 0,
                'e1': e1_lens, 
                'e2': e2_lens, 
                'gamma': example_lens['lens_mass_gamma'],}

psi_ext, gamma_ext = param_util.ellipticity2phi_gamma(e1=example_lens['ext_shear_e1'],
                                                      e2=example_lens['ext_shear_e2'])
kwargs_shear = {'gamma_ext': -gamma_ext,  # Note the minus
                'psi_ext': psi_ext + 0.5*np.pi # Note the added term
               }

kwargs_ps = {'ra_source': example_lens['host_pos'][0],
              'dec_source': example_lens['host_pos'][1],}

measured_vd = example_lens['measured_vel_disp']
measured_vd_err = example_lens['measured_vel_disp_err']

measured_td = np.array(example_lens['measured_time_delays'])
measured_td_err = np.array(example_lens['measured_time_delays_err'])

kwargs_model = {'lens_model_list': ['SPEMD', 'SHEAR_GAMMA_PSI'],
                'lens_light_model_list': ['SERSIC_ELLIPSE'],
                'source_light_model_list': ['SERSIC_ELLIPSE'],
                'point_source_model_list': ['SOURCE_POSITION'],
               #'point_source_model_list' : ['LENSED_POSITION']
               }

r_eff = example_lens['lens_light_R_sersic']

post = h0rton.h0_inference.H0Posterior(z_lens=example_lens['z_lens'],
                                       z_src=example_lens['z_src'],
                                       lens_mass_dict=kwargs_spemd,
                                       ext_shear_dict=kwargs_shear,
                                       ps_dict=kwargs_ps,
                                       measured_vd=measured_vd,
                                       measured_vd_err=measured_vd_err,
                                       measured_td=measured_td,
                                       measured_td_err=measured_td_err,
                                       lens_light_R_sersic=r_eff,
                                       H0_prior=uniform(loc=50, scale=40),
                                       kappa_ext_prior=norm(loc=example_lens['kappa_ext'], scale=1.e-6),
                                       aniso_param_prior=uniform(loc=0.5*r_eff, scale=4.5*r_eff),
                                       kwargs_model=kwargs_model,
                                       abcd_ordering_i=[0, 1, 2, 3],
                                      )

In [ ]:
samples_dict = post.get_samples(10000, seed=123, exclude_vel_disp=True)

In [ ]:
plt.hist(samples_dict['H0_samples'],
         weights=samples_dict['H0_weights'], bins=50, range=[70, 80], density=True, label='samples')
plt.axvline(example_lens['H0'], color='r', label='truth')
plt.title(example_lens['name'])
plt.xlabel('H0')
plt.ylabel('density')
plt.legend()

Did we order the image positions correctly? It seems so, as the inferred image positions (red) and the published image positions (blue) lie on top of one another.

In [ ]:
pixel_scale = 0.08 # after drizzle
n_pix = 99//2

plt.matshow(img, extent=[-n_pix*pixel_scale, n_pix*pixel_scale, -n_pix*pixel_scale, n_pix*pixel_scale], 
            origin='lower left', norm=asinh_norm)
plt.plot(example_lens['agn_img_pos_x'],
         example_lens['agn_img_pos_y'], 'b.')
plt.plot(post.reorder_to_tdlmc(post.kwargs_ps[0]['ra_image']),
         post.reorder_to_tdlmc(post.kwargs_ps[0]['dec_image']), 'r.')

You can also check the offset in arcsec, to ensure that you ordered the image positions and time delays to agree with TDLMC's. You can check that the offste is within a few milliarcseconds.

In [ ]:
post.calculate_offset_from_true_image_positions(example_lens['agn_img_pos_x'],
                                                example_lens['agn_img_pos_y'])

The below cell demonstrates how to define the point source in terms of the image positions instead, although this is not recommended.

In [ ]:
e1_lens, e2_lens = param_util.phi_q2_ellipticity(q=example_lens['lens_mass_q'], 
                                                 phi=example_lens['lens_mass_phi_G'])
kwargs_spemd = {'theta_E': example_lens['lens_mass_theta_E'],
                'center_x': 0, 
                'center_y': 0,
                'e1': e1_lens, 
                'e2': e2_lens, 
                'gamma': example_lens['lens_mass_gamma'],}

psi_ext, gamma_ext = param_util.ellipticity2phi_gamma(e1=example_lens['ext_shear_e1'],
                                                      e2=example_lens['ext_shear_e2'])
kwargs_shear = {'gamma_ext': -gamma_ext,  # Note the minus
                'psi_ext': psi_ext + 0.5*np.pi # Note the added term
               }

kwargs_ps = {'ra_image': example_lens['agn_img_pos_x'],
              'dec_image': example_lens['agn_img_pos_y'],}

measured_vd = example_lens['measured_vel_disp']
measured_vd_err = example_lens['measured_vel_disp_err']

measured_td = np.array(example_lens['measured_time_delays'])
measured_td_err = np.array(example_lens['measured_time_delays_err'])

kwargs_model = {'lens_model_list': ['SPEMD', 'SHEAR_GAMMA_PSI'],
                'lens_light_model_list': ['SERSIC_ELLIPSE'],
                'source_light_model_list': ['SERSIC_ELLIPSE'],
                #'point_source_model_list': ['SOURCE_POSITION'],
               'point_source_model_list' : ['LENSED_POSITION']
               }

r_eff = example_lens['lens_light_R_sersic']

post_published_img = h0rton.h0_inference.H0Posterior(z_lens=example_lens['z_lens'],
                                       z_src=example_lens['z_src'],
                                       lens_mass_dict=kwargs_spemd,
                                       ext_shear_dict=kwargs_shear,
                                       ps_dict=kwargs_ps,
                                       measured_vd=measured_vd,
                                       measured_vd_err=measured_vd_err,
                                       measured_td=measured_td,
                                       measured_td_err=measured_td_err,
                                       lens_light_R_sersic=r_eff,
                                       H0_prior=uniform(loc=50, scale=40),
                                       kappa_ext_prior=norm(loc=0.0, scale=0.025),
                                       aniso_param_prior=uniform(loc=0.5*r_eff, scale=4.5*r_eff),
                                       kwargs_model=kwargs_model,
                                       abcd_ordering_i=[0, 1, 3, 2],
                                      )

By definition, the offset from the true image positions should be 0 arcsec for each image.

In [ ]:
post_published_img.calculate_offset_from_true_image_positions(example_lens['agn_img_pos_x'],
                                                        example_lens['agn_img_pos_y'])

In [ ]:
samples_published_img_dict = post_true_img.get_samples(10000, seed=123, exclude_vel_disp=True)

In [ ]:
plt.hist(samples_published_img_dict['H0_samples'],
         weights=samples_published_img_dict['H0_weights'], bins=25, density=True, label='samples')
plt.axvline(example_lens['H0'], color='r', label='truth')
plt.title(example_lens['name'])
plt.xlabel('H0')
plt.ylabel('density')
plt.legend()